## Process of MSE implementing 

our currect distribution sample: 

ic| normalized_counts: [0.057,
                        0.068,
                        0.051,
                        0.072,
                        0.071,
                        0.095,
                        0.043,
                        0.080,
                        0.049,
                        0.062,
                        0.066,
                        0.061,
                        0.070,
                        0.059,
                        0.053,
                        0.043]

uniform_count: 0.0625

Loss function design problem. 

Background:
    We need to include unlabelled data into our model to maximize accuracy of predicting 16 classes of our system. To acheieve this goal we pretrained simple Encoder-Decoder model on our labelled data (91-92% accuracy). Now, our job is set up a programm which will use pretrain network for inicial classificationo with following LOSS which will adjust NEW model to classify unlabelled data more accuratly. 

    So far we tried multiple different ways to make it work. MSE loss, CrossEntropy, addition of gumble softmax function, and finally Correlation loss with multiple modification and changes. 


Loss isn't working in any of the cases. Originally I had problems with discritness of Counter function needed for MSE (we can't use threshold method or counter). Gumble sofmax with correlation loss fixed the problem with discritness and step function, but loss didn't allow the network to learn all the states, we arrived to binary scenario, where we either learn that all states are 0000 or 1111. 

Problem statement:

What are we doing is, we pass unlabelled data to our pretrained model and receive predictions. We want to construct a disctribution of 16 unique states we have predicted and construct loss to push it as close as possible to uniform. 

in code we have "samples" as an input from our pre-saved model classification. we need to pass these predictions to the loss function.

Input new_model receives (first 10 samples) from an old pretrained:

tensor([[[0.5724], 
         [0.1302], 
         [0.2840], 
         [0.7402]],

        [[0.9580], 
         [0.9998], 
         [0.9998], 
         [1.0000]],

        [[0.9959], 
         [0.3668], 
         [0.9714], 
         [0.1976]],

        [[0.9451], 
         [0.9999], 
         [0.2559], 
         [0.1247]],

        [[0.5298], 
         [0.1069], 
         [0.4578],
         [0.0506]],

        [[0.9986],
         [0.0427],
         [0.1707],
         [0.0350]],

        [[0.9998],
         [0.7080],
         [0.9746],
         [0.9995]],

        [[0.0520],
         [0.0258],
         [0.4781],
         [0.0730]],

        [[0.0538],
         [0.1909],
         [0.3127],
         [0.1140]],

        [[0.3626],
         [0.9971],
         [0.1013],
         [0.7093]]], grad_fn=<SliceBackward0>)

We used F.gumbel_softmax to avoid step function (which is discrete) and get this output before passing it to our loss. But this approach could be false. 



Gumbel samples: 
tensor([[[1., 0.],
         [1., 0.],
         [1., 0.],
         [0., 1.]],

        [[0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.]],

        [[0., 1.],
         [1., 0.],
         [0., 1.],
         [1., 0.]],

        [[0., 1.],
         [0., 1.],
         [1., 0.],
         [1., 0.]],

        [[0., 1.],
         [1., 0.],
         [0., 1.],
         [1., 0.]],

        [[0., 1.],
         [0., 1.],
         [1., 0.],
         [1., 0.]],

        [[0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.]],

        [[1., 0.],
         [1., 0.],
         [1., 0.],
         [1., 0.]],

        [[1., 0.],
         [1., 0.],
         [1., 0.],
         [1., 0.]],

        [[1., 0.],
         [0., 1.],
         [1., 0.],
         [1., 0.]]], grad_fn=<SliceBackward0>)

In [1]:
step = 0
for epoch in range(N_epochs):
    running_loss = 0.0
    for i, xbatch in enumerate(halfpi_train_loader):  # Adjusted for unlabelled data
        optimizer.zero_grad()
        xbatch = xbatch.to(device=device)

        # Ensure the input tensor has requires_grad=True
        xbatch.requires_grad_(True)

        batch_loss = corrloss(xbatch)
        batch_loss.backward()

        running_loss += (
            batch_loss.item()
        )  # Using .item() to avoid accumulation of the graph

        optimizer.step()
    step += 1

    running_loss /= i + 1

    schedule.step()

    if epoch % log_every == 0 or epoch == N_epochs - 1:
        val_loss = torch.tensor(
            [
                corrloss(
                    databatch.to(device=device)
                )  # Call the corrloss function directly
                for databatch in halfpi_val_loader
            ]
        ).mean()

        print(
            "{:<100}".format(
                "\r"
                + "[{:<60}] ".format(
                    "=" * ((np.floor((epoch + 1) / N_epochs * 60)).astype(int) - 1)
                    + ">"
                    if epoch + 1 < N_epochs
                    else "=" * 60
                )
                + "{:<40}".format(
                    "Epoch {}/{}: Loss(Train) = {:.3f}, Loss(Val) = {:.3f}".format(
                        epoch + 1, N_epochs, running_loss, val_loss
                    )
                )
            ),
            end="",
        )
        sys.stdout.flush()

NameError: name 'N_epochs' is not defined

In [ ]:
def get_state_counts(predictions, num_states=16):

    binary_preds = (predictions > 0.5).int()
    binary_strings = [
        "".join(map(str, preds)) for preds in binary_preds.cpu().numpy().reshape(-1, 4)
    ]
    tensor_str_counts = Counter(binary_strings)
    # counts to probs
    counts = [tensor_str_counts.get(f"{i:04b}", 0) for i in range(num_states)]
    probabilities = torch.tensor(counts, dtype=torch.float32) / len(binary_strings)
    return probabilities


def kldiv_loss(predictions, num_classes):
    predicted_probs = get_state_counts(predictions, num_classes)

    uniform_probs = torch.full_like(predicted_probs, 1.0 / num_classes)

    # Ensure probs are non-zero
    predicted_probs = torch.clamp(predicted_probs, min=1e-10)
    uniform_probs = torch.clamp(uniform_probs, min=1e-10)

    # Compute KL
    kl_div = F.kl_div(predicted_probs.log(), uniform_probs, reduction="batchmean")

    return kl_div

In [ ]:
## Hyper parameters optimization block: 
log_every = 1

########################################################################################
# Best hyperparameters:  {'lr': 0.006185526944778624, 'weight_decay': 0.009880060840556128}

def objective(trial):
    N_epochs = 50
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1)

    model = MultiIonReadout(encoder, classifier).to(device)
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    schedule_params = {"factor": 1}
    schedule = lr_scheduler.ConstantLR(optimizer, **schedule_params)

    step = 0
    for epoch in range(N_epochs):
        running_loss = 0.0
        for i, databatch in enumerate(train_loader):
            optimizer.zero_grad()
            (xbatch, ybatch) = databatch
            xbatch = xbatch.to(device=device)
            ybatch = ybatch.to(device=device)

            batch_loss = model.train().nllloss(xbatch, ybatch)
            batch_loss.backward()

            running_loss += batch_loss.item()

            optimizer.step()
        step += 1

        running_loss /= i + 1

        schedule.step()

        if epoch % log_every == 0 or epoch == N_epochs - 1:
            val_loss = (
                torch.tensor(
                    [
                        model.eval().nllloss(
                            databatch[0].to(device=device),
                            databatch[1].to(device=device),
                        )
                        for databatch in val_loader
                    ]
                )
                .mean()
                .item()
            )

            sia = (
                torch.tensor(
                    [
                        model.eval().sia(
                            databatch[0].to(device=device),
                            databatch[1].to(device=device),
                        )
                        for databatch in val_loader
                    ]
                )
                .mean()
                .item()
            )

            print(
                "{:<180}".format(
                    "\r"
                    + "[{:<60}] ".format(
                        "=" * ((np.floor((epoch + 1) / N_epochs * 60)).astype(int) - 1)
                        + ">"
                        if epoch + 1 < N_epochs
                        else "=" * 60
                    )
                    + "{:<40}".format(
                        "Epoch {}/{}: NLL Loss(Train) = {:.3g}, NLL Loss(Val) = {:.3g}, Accuracy(Val) = {:.3f}".format(
                            epoch + 1, N_epochs, running_loss, val_loss, sia
                        )
                    )
                ),
                end="",
            )
            sys.stdout.flush()

    return val_loss


# Creating a study object and optimize the objective function
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Print the best hyperparameters
print(f"Best trial: {study.best_trial.value}")
print("Best hyperparameters: ", study.best_trial.params)

In [ ]:
# Best hyperparameters:  {'lr': 0.00016388181712790806, 'weight_decay': 4.6547937254492916e-05}
def objective(trial):
    N_epochs = 20
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-5, 1)

    optimizer = Adam(enhanced_model.parameters(), lr=lr, weight_decay=weight_decay)
    schedule_params = {"factor": 1}
    schedule = lr_scheduler.ConstantLR(optimizer, **schedule_params)
    log_every = 1

    step = 0
    for epoch in range(N_epochs):
        running_loss = 0.0
        for i, xbatch in enumerate(halfpi_train_loader):  # Correct unpacking
            optimizer.zero_grad()
            xbatch = xbatch.to(device=device)
            xbatch.requires_grad_(True)

            batch_loss = enhanced_model.train().corrloss(xbatch)
            batch_loss.backward()

            if batch_loss.isnan():
                break

            running_loss += batch_loss.item()

            optimizer.step()
        step += 1

        running_loss /= i + 1

        schedule.step()

        if epoch % log_every == 0 or epoch == N_epochs - 1:
            val_loss = (
                torch.tensor(
                    [
                        enhanced_model.eval().corrloss(databatch.to(device=device))
                        for databatch in halfpi_val_loader
                    ]
                )
                .mean()
                .item()
            )

            sia = (
                torch.tensor(
                    [
                        enhanced_model.eval().sia(
                            databatch[0].to(device=device),
                            databatch[1].to(device=device),
                        )
                        for databatch in val_loader
                    ]
                )
                .mean()
                .item()
            )

            print(
                "{:<180}".format(
                    "\r"
                    + "[{:<60}] ".format(
                        "=" * ((np.floor((epoch + 1) / N_epochs * 60)).astype(int) - 1)
                        + ">"
                        if epoch + 1 < N_epochs
                        else "=" * 60
                    )
                    + "{:<40}".format(
                        "Epoch {}/{}: Correlation Loss(Train) = {:.3f}, Correlation Loss(Val) = {:.3f}, Accuracy(All Bright & Dark) = {:.3f}".format(
                            epoch + 1, N_epochs, running_loss, val_loss, sia
                        )
                    )
                ),
                end="",
            )
            sys.stdout.flush()

    return val_loss


# Create a study object and optimize the objective function
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Print the best hyperparameters
print(f"Best trial: {study.best_trial.value}")
print("Best hyperparameters: ", study.best_trial.params)

In [2]:
## new structure of our dataset:

Label 0: [[0.]
 [0.]
 [0.]
 [0.]]
Measurement 0: [[201. 198. 202. 201. 198. 201. 203. 203. 201. 210. 201. 199. 199. 202.
  198. 201. 199. 201. 204. 196. 205. 198. 201. 201. 202.]
 [200. 202. 194. 208. 208. 198. 199. 202. 204. 198. 202. 201. 202. 198.
  199. 198. 196. 199. 201. 202. 208. 203. 199. 198. 202.]
 [201. 201. 202. 199. 200. 217. 203. 199. 202. 201. 202. 201. 206. 201.
  202. 207. 199. 199. 198. 200. 198. 200. 203. 200. 200.]
 [198. 199. 200. 198. 200. 201. 199. 206. 201. 196. 201. 200. 198. 201.
  200. 201. 199. 203. 204. 199. 199. 201. 202. 209. 203.]]
Label 1: [[0.]
 [0.]
 [0.]
 [0.]]
Measurement 1: [[199. 200. 200. 203. 204. 202. 199. 200. 197. 206. 200. 200. 199. 201.
  201. 198. 202. 212. 201. 201. 195. 199. 198. 203. 199.]
 [203. 200. 195. 209. 200. 198. 200. 200. 202. 198. 201. 200. 198. 199.
  201. 196. 201. 201. 198. 197. 202. 200. 199. 199. 193.]
 [203. 201. 201. 202. 198. 210. 200. 200. 201. 202. 200. 200. 201. 201.
  198. 202. 209. 199. 200. 198. 201. 203. 200. 200. 206.]
 [201. 201. 200. 207. 201. 204. 203. 198. 201. 200. 199. 200. 198. 197.
  200. 199. 199. 198. 202. 203. 198. 199. 202. 206. 200.]]
Label 2: [[0.]
...
 [199. 199. 203. 199. 196. 208. 200. 206. 200. 199. 201. 198. 200. 202.
  198. 201. 198. 199. 201. 200. 198. 201. 200. 201. 198.]
 [199. 201. 203. 198. 200. 201. 202. 203. 202. 207. 201. 198. 196. 201.
  195. 201. 201. 198. 198. 202. 197. 199. 202. 202. 200.]]

SyntaxError: invalid syntax (731669184.py, line 3)